In [11]:
# -------- Imports --------
import sys
import os
import numpy as np
import scipy.sparse as sp
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))
from _utility import *


In [ ]:
# -------- Parameters --------
# -- Grid parameters --
Nx, Ny  = 200, 200
dx, dy = 1.0, 1.0
psi_len = Nx*Ny + (Nx-1)*Ny + Nx*(Ny-1) # Number of values for the staggered grid

# -- Wave field definition --
# Pressure field (u)
f0 = 0.5            # Central frequency of the Ricker wavelet
x0, y0 = 0.0, 0.0  # Wavelet center
X, Y = np.meshgrid(np.linspace(-1, 1, Nx), np.linspace(-1, 1, Ny))
u0 = Ricker(f0, X, Y, 0, x0/Nx, y0/Ny)

# Particle velocity field x (vx)
v0x = np.zeros((Ny, (Nx-1)))

# Particle velocity field y (vy)
v0y = np.zeros(((Ny-1), Nx))

# Stack the wave field components
phi_0 = np.hstack([u0.flatten(), v0x.flatten(), v0y.flatten()])

# -- Material properties --
# Velocity (c)
c0 = 3
c_model = c0 * np.ones((Ny, Nx))

# Density (rho)
rho0 = 2
rho_model = rho0 * np.ones((Ny, Nx))
rho_stag_x = rho0 * np.ones((Ny, (Nx-1)))
rho_stag_y = rho0 * np.ones(((Ny-1), Nx))

# -- Subspace projector --
mask = np.random.choice([0, 1], size=psi_len, p=[0.5, 0.5])
P = sp.diags(mask)

# -- "Projection" unitary --
# Single state (Energy estimate)
P_hat_1 = sp.block_array([
    [P, sp.eye(psi_len) - P],
    [sp.eye(psi_len) - P, P]
    ])

# Two-state (L2 estimate)
Z = sp.csr_array(P.shape)
P_hat_2 = sp.block_array([[P, Z, sp.eye(psi_len)-P, Z], 
                         [Z, P, Z, sp.eye(psi_len)-P],
                         [sp.eye(psi_len)-P, Z, P, Z],
                         [Z, sp.eye(psi_len)-P, Z, P]])

# Check if the "projectors" are unitary
print("Unitary P_hat_1:", not (P_hat_1 @ P_hat_1.conj().T != sp.eye(2*psi_len)).sum())
print("Unitary P_hat_2:", not (P_hat_1 @ P_hat_1.conj().T != sp.eye(2*psi_len)).sum())


Unitary P_hat_1: True
Unitary P_hat_2: True


In [13]:
# -------- Material Transform (Acoustic) --------
(B, B_sqrt, B_inv, _) = compute_B(c_model, rho_model, rho_stag_x, rho_stag_y)


In [14]:
# -------- Loss function 1: Subspace Energy (one state) --------
# Normalize the state and transform it to the energy basis (quantum state)
psi_0 = B_sqrt @ phi_0
norm = np.linalg.norm(psi_0)
psi_0 /= norm

# Expand the state with auxliary dimension (doubling the size)
psi_0 = np.hstack([psi_0, np.zeros_like(psi_0)])

# Compute the ground-truth energy (uniform element volume)
EN_GT = (1/2) * np.linalg.norm(B_sqrt @ (P @ phi_0))**2 * (dx * dy)
print('Ground-Truth Energy:', EN_GT.round(4))

# Define quantum observable (Efficient implementation, 2 Paulis)
O_EN = sp.block_array([
    [sp.eye(psi_len), None,],
    [None, sp.csr_matrix((psi_len, psi_len))]
])
print('Hermitian Observable:', not (O_EN - O_EN.conj().T).nnz)

# Apply unitary "projection" transform (quantum gates)
psi_0 = P_hat_1 @ psi_0

# Compute expectation value (quantum measurement)
E_EN = np.abs(psi_0.conj().T @ O_EN @ psi_0)
print('Expectation Value:', E_EN.round(4))

# Compute the energy loss (post-processing)
EN_QC = (1/2) * E_EN * norm**2 * (dx * dy)
print('Energy Estimate:', EN_QC.round(4))


Ground-Truth Energy: 78.437
Hermitian Observable: True
Expectation Value: 0.4964
Energy Estimate: 78.437


In [15]:
# -------- Loss function 2: Subspace Energy-transformed L2 Distance  (two states) --------
# Add a comparison state
rand = (np.random.rand(psi_len) - 0.5) * 1
phi_c = P @ (phi_0 + rand)
phi_l2 = np.hstack([phi_0, phi_c])

# Normalize the state and transform it to the energy basis (quantum state)
psi_l2 = sp.block_diag(2*[B_sqrt]) @ phi_l2
norm = np.linalg.norm(psi_l2)
psi_l2 /= norm

# Expand the state with auxliary dimension (doubling the size)
psi_l2 = np.hstack([psi_l2, np.zeros_like(psi_l2)])

# Compute the ground-truth L2 distance
L2_GT = np.linalg.norm(B_sqrt @ (P @ phi_0) - B_sqrt @ phi_c)
print('Ground-Truth L2 Distance:', L2_GT.round(4))

# Define quantum observable
O_L2 = sp.block_array([[sp.eye(psi_len), -sp.eye(psi_len)],
                       [-sp.eye(psi_len), sp.eye(psi_len)]])
O_L2 = sp.block_diag([O_L2, sp.csr_matrix((2*psi_len, 2*psi_len))])
print('Hermitian Observable:', not (O_L2 - O_L2.conj().T).nnz)

# Apply unitary "projection" transform (quantum gates)
psi_l2 = P_hat_2 @ psi_l2

# Compute expectation value (quantum measurement)
E_L2 = np.abs(psi_l2.conj().T @ O_L2 @ psi_l2)
print('Expectation Value:', E_L2.round(4))

# Compute the l2 loss (post-processing)
L2_QC = np.sqrt(E_L2) * norm
print('L2 Estimate:', L2_QC.round(4))


Ground-Truth L2 Distance: 81.9755
Hermitian Observable: True
Expectation Value: 0.9339
L2 Estimate: 81.9755
